# Regularización Elastic Net para Predicción de Humedad (_humedad)
Este notebook implementa un modelo de regresión con regularización Elastic Net para predecir la humedad, con todas las curvas y métricas.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split, learning_curve, validation_curve, cross_val_score
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Cargar el dataset
dataset = pd.read_csv('../dataset/ML1_temperature.xlsx - Hoja1.csv', decimal=',')
x = dataset.drop(['recnt_Humidity','recnt_Temperature'], axis=1).values
y = dataset['recnt_Humidity'].values.reshape(-1, 1)

In [ ]:
# Separar en entrenamiento y validación
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

## Curva de aprendizaje
Muestra cómo varía el error (MSE) al predecir humedad según el tamaño del conjunto de entrenamiento.

In [ ]:
train_sizes, train_scores, val_scores = learning_curve(ElasticNet(alpha=1.0, l1_ratio=0.5), x_train, y_train.ravel(), cv=5, scoring='neg_mean_squared_error')
plt.plot(train_sizes, -train_scores.mean(axis=1), label='Entrenamiento')
plt.plot(train_sizes, -val_scores.mean(axis=1), label='Validación')
plt.xlabel('Tamaño del conjunto de entrenamiento')
plt.ylabel('MSE (Humedad)')
plt.title('Curva de Aprendizaje - Elastic Net (Humedad)')
plt.legend()
plt.grid(True)
plt.show()

## Curva de validación
Muestra el desempeño del modelo para diferentes valores de alpha al predecir humedad.

In [ ]:
alphas = np.logspace(-4, 1, 10)
train_scores, val_scores = validation_curve(ElasticNet(l1_ratio=0.5), x_train, y_train.ravel(), param_name='alpha', param_range=alphas, cv=5, scoring='neg_mean_squared_error')
plt.semilogx(alphas, -train_scores.mean(axis=1), label='Entrenamiento')
plt.semilogx(alphas, -val_scores.mean(axis=1), label='Validación')
plt.xlabel('Alpha')
plt.ylabel('MSE (Humedad)')
plt.title('Curva de Validación - Elastic Net (Humedad)')
plt.legend()
plt.grid(True)
plt.show()

## Curva de paridad
Compara las predicciones de humedad con los valores reales.

In [ ]:
# Curva de paridad mejorada: Humedad Real vs Predicha (Entrenamiento y Validación)
plt.figure(figsize=(7,6))
plt.scatter(y_train, model.predict(x_train), alpha=0.7, label='Entrenamiento', color='cornflowerblue')
plt.scatter(y_val, y_val_pred, alpha=0.7, label='Validación', color='darkorange')
plt.plot([min(y), max(y)], [min(y), max(y)], 'r--', label='Predicción perfecta')
plt.xlabel('Humedad Real')
plt.ylabel('Humedad Predicha')
plt.title(f'Humedad Real vs Predicha\nMSE Validación = {mean_squared_error(y_val, y_val_pred):.3f}')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
print(f'R^2 validación (Humedad): {r2_score(y_val, y_val_pred):.3f}')

## Curva de sesgo y varianza
Analiza la diferencia entre el error de entrenamiento y validación al predecir humedad.

In [ ]:
y_train_pred = model.predict(x_train)
print(f'MSE Entrenamiento (Humedad): {mean_squared_error(y_train, y_train_pred):.3f}')
print(f'MSE Validación (Humedad): {mean_squared_error(y_val, y_val_pred):.3f}')
print(f'R^2 Entrenamiento (Humedad): {r2_score(y_train, y_train_pred):.3f}')
print(f'R^2 Validación (Humedad): {r2_score(y_val, y_val_pred):.3f}')

## Curva de validación cruzada
Evalúa el desempeño promedio del modelo usando validación cruzada al predecir humedad.

In [ ]:
cv_scores = cross_val_score(ElasticNet(alpha=1.0, l1_ratio=0.5), x, y.ravel(), cv=5, scoring='r2')
print(f'R^2 promedio validación cruzada (Humedad): {cv_scores.mean():.3f} ± {cv_scores.std():.3f}')

## Explicación breve
Elastic Net permite predecir la humedad combinando las ventajas de Ridge y Lasso. El compromiso sesgo-varianza se ajusta modificando los parámetros alpha y l1_ratio.

In [ ]:
# Actual vs Predicho (Entrenamiento y Validación) - Elastic Net (Humedad)
plt.figure(figsize=(12, 6))
plt.scatter(y_train, y_train_pred, alpha=0.5, label='Entrenamiento')
plt.scatter(y_val, y_val_pred, alpha=0.5, label='Validación')
plt.xlabel('Humedad Real')
plt.ylabel('Humedad Predicha')
plt.title('Actual vs Predicho - Elastic Net (Humedad)')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Serie temporal: valores reales vs predichos
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

plt.figure(figsize=(10,4))
plt.plot(range(len(y_val)), y_val, label='Real', color='royalblue', linewidth=2)
plt.plot(range(len(y_val_pred)), y_val_pred, label='Predicho', color='darkorange', linewidth=2)
plt.xlabel('Índice (tiempo)')
plt.ylabel('Variable objetivo')
plt.title(f'Serie temporal: Real vs Predicho (R²={r2_score(y_val, y_val_pred):.3f})')
plt.legend()
plt.tight_layout()
plt.show()